In [75]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

In [76]:
df= pd.read_csv(r"D:\DATA\Data\U\02 Python\02 Project\CV\RFM\rfm_data\rfm_data.csv")
print(df.head())
from datetime import datetime

   CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0        8814   2023-04-11             943.31          Product C   890075   
1        2188   2023-04-11             463.70          Product A   176819   
2        4608   2023-04-11              80.28          Product A   340062   
3        2559   2023-04-11             221.29          Product A   239145   
4        9482   2023-04-11             739.56          Product A   194545   

   Location  
0     Tokyo  
1    London  
2  New York  
3    London  
4     Paris  


In [77]:
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


In [79]:
df['PurchaseDate']=pd.to_datetime(df['PurchaseDate'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   int64         
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   int64         
 5   Location            1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 47.0+ KB


In [80]:
df.describe()

,CustomerID,PurchaseDate,TransactionAmount,OrderID
count,1000.000000,1000,1000.00000,1000.000000
mean,5554.789000,2023-05-10 07:27:50.400000,513.67781,554071.398000
min,1011.000000,2023-04-11 00:00:00,12.13000,100096.000000
25%,3273.000000,2023-04-26 00:00:00,257.12000,313152.000000
50%,5538.000000,2023-05-09 00:00:00,523.56500,564671.500000
75%,7821.750000,2023-05-26 00:00:00,759.86000,783052.250000
max,9991.000000,2023-06-10 00:00:00,999.44000,999695.000000
std,2605.014863,NaN,286.09870,264695.448814


In [81]:
df.isnull().sum()

CustomerID            0
PurchaseDate          0
TransactionAmount     0
ProductInformation    0
OrderID               0
Location              0
dtype: int64

In [82]:
df['Location'].value_counts()

Location
Tokyo       278
New York    247
London      246
Paris       229
Name: count, dtype: int64

In [83]:
df['ProductInformation'].value_counts()

ProductInformation
Product C    277
Product D    253
Product B    245
Product A    225
Name: count, dtype: int64

In [84]:
# Convert 'PurchaseDate' to datetime
df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])

In [85]:
# Calculate Recency
df['Recency'] = (datetime.now() - df['PurchaseDate']).dt.days

In [86]:
# Calculate Frequency
frequency_data = df.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_data.rename(columns={'OrderID': 'Frequency'}, inplace=True)
df = df.merge(frequency_data, on='CustomerID', how='left')

In [87]:

# Calculate Monetary Value
monetary_data = df.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_data.rename(columns={'TransactionAmount': 'MonetaryValue'}, inplace=True)
df = df.merge(monetary_data, on='CustomerID', how='left')

In [88]:
import numpy as np
from scipy.stats import gaussian_kde
# data
x=df['Recency']
# Caculate KDE
kde=gaussian_kde(x)
x_range=np.linspace(x.min(),x.max(),200)
y_kde=kde(x_range)

#Create Figure
fig=go.Figure()
# Histogram count
# Histogram count
fig.add_trace(go.Histogram(
    x=x,
    nbinsx=20,
    name="Customer Count",
    marker_color='skyblue',
    opacity=0.7
))

#KDE Chart
fig.add_trace(go.Scatter(
    x=x_range,
    y=y_kde*len(x)*((x.max()-x.min())/20),
    name="Recency Density (KDE)",
))
fig.update_layout(
    yaxis=dict(title='Customer_Count'),
    title="Histogram of Recency with KDE Density Curve",
    barmode="overlay"
)

In [89]:
freq=df['Frequency']
q80=np.quantile(freq,0.8)
med=np.median(freq)
fig=px.histogram(df,
                x='Frequency',
                nbins=30,
                marginal='box',
                title='How often do the customer buy (Spot the loyalist)',
                labels={'Frequecy':'Purchases per Customer'})
fig.add_vline(x=med,line_dash='dash',line_width=2,
              annotation_text=f'Median: {med}',annotation_position='top left')
fig.add_vrect(x0=q80,x1=freq.max(), opacity=0.1, line_width=0,
              annotation_text='Top 20% loyalists', annotation_position='top right')
fig.add_annotation(x=freq.min(),yshift=40,text='Left tail: one-time/rare buyers → retention opportunities')

In [90]:
# Define scoring criteria for each RFM value
recency_scores = [5, 4, 3, 2, 1]  
frequency_scores = [1, 2, 3, 4, 5]  
monetary_scores = [1, 2, 3, 4, 5]  

# Calculate RFM scores
df['RecencyScore'] = pd.cut(df['Recency'], bins=5, labels=recency_scores)
df['FrequencyScore'] = pd.cut(df['Frequency'], bins=5, labels=frequency_scores)
df['MonetaryScore'] = pd.cut(df['MonetaryValue'], bins=5, labels=monetary_scores)

In [91]:
# Convert RFM scores to numeric type
df['RecencyScore'] = df['RecencyScore'].astype(int)
df['FrequencyScore'] = df['FrequencyScore'].astype(int)
df['MonetaryScore'] = df['MonetaryScore'].astype(int)

In [92]:
# Calculate RFM score 
df['RFM_Score'] = df['RecencyScore'] + df['FrequencyScore'] + df['MonetaryScore']

In [93]:
# Create RFM segments based on the RFM score
segment_labels = ['Low-Value', 'Medium-Value', 'High-Value']
df['Value Segment'] = pd.qcut(df['RFM_Score'], q=3, labels=segment_labels)

In [94]:
# RFM Segment Distribution
segment_counts = df['Value Segment'].value_counts().reset_index()
segment_counts.columns = ['Value Segment', 'Count']

In [95]:
pastel_colors = px.colors.qualitative.Pastel

# Create the bar chart
segment_chart= px.bar(segment_counts, x='Value Segment', y='Count', 
                          color='Value Segment', color_discrete_sequence=pastel_colors,
                          title='RFM Value Segment Distribution')

# Update the layout
segment_chart.update_layout(xaxis_title='RFM Value Segment',
                              yaxis_title='Count',
                              showlegend=False)

# Show the figure
segment_chart.show()

RFM Customer Segments

In [96]:
# Assign RFM segments based on the RFM score
def assign_segment(x):
    if x>=9:
        return "Champions"
    elif x>=6:
        return 'Potential Loyalists'
    elif x>=5:
        return 'At Risk Customers'
    elif x>=4:
        return "Can't Lose"
    elif x >=3:
        return "Lost"
    else: 
        return "Others"

df['RFM Customer Segments'] =df['RFM_Score'].apply(assign_segment)
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score,Value Segment,RFM Customer Segments
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,878,1,943.31,1,1,2,4,Low-Value,Can't Lose
1,2188,2023-04-11,463.70,Product A,176819,London,878,1,463.70,1,1,1,3,Low-Value,Lost
2,4608,2023-04-11,80.28,Product A,340062,New York,878,1,80.28,1,1,1,3,Low-Value,Lost
3,2559,2023-04-11,221.29,Product A,239145,London,878,1,221.29,1,1,1,3,Low-Value,Lost
4,9482,2023-04-11,739.56,Product A,194545,Paris,878,1,739.56,1,1,2,4,Low-Value,Can't Lose


In [112]:
seg=df['RFM Customer Segments'].value_counts().reset_index()
seg['Share']=(seg['count']/(seg['count'].sum()))*100
seg

,RFM Customer Segments,count,Share
0,Potential Loyalists,503,50.3
1,At Risk Customers,180,18.0
2,Can't Lose,173,17.3
3,Lost,82,8.2
4,Champions,62,6.2


In [114]:
import plotly.colors

pastel_colors = plotly.colors.qualitative.Pastel

segment_counts = df['RFM Customer Segments'].value_counts()

# Create a bar chart to compare segment counts
fig = go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                            marker=dict(color=pastel_colors))])

# Set the color of the Champions segment as a different color
champions_color = 'rgb(158, 202, 225)'
fig.update_traces(marker_color=[champions_color if segment == 'Champions' else pastel_colors[i]
                                for i, segment in enumerate(segment_counts.index)],
                  marker_line_color='rgb(8, 48, 107)',
                  marker_line_width=1.5, opacity=0.6)

# Update the layout
fig.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)

fig.show()

RFM Analysis

In [ ]:
segment_product_counts = df.groupby(['Value Segment','RFM Customer Segments']).size().reset_index(name='Count')\
                        .sort_values('Count',ascending=False)
segment_product_counts.head()


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12056\2759657211.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,Value Segment,RFM Customer Segments,Count
9,Medium-Value,Potential Loyalists,386
0,Low-Value,At Risk Customers,180
1,Low-Value,Can't Lose,173
14,High-Value,Potential Loyalists,117
3,Low-Value,Lost,82


Champions: Your best customers. They often buy and spend the most.  
Loyal Customers: High Frequency and monetary value, but their recency might be slightly lower than that of champions.  
New Customers: Bought recently but haven’t purchased frequently or spent much.  
Lost Customers: Haven’t bought in a long time, don’t buy often, and don’t spend much.  

In [ ]:
tree_map_segment_products=px.treemap(
                            segment_product_counts,path=['Value Segment','RFM Customer Segments'],
                            values='Count',
                            color='Value Segment',
                            title='RFM Customer Segments by Value',
                            color_discrete_sequence=px.colors.qualitative.Pastel
)
tree_map_segment_products.show()

In [131]:
# Filter the data to include only the customers in the Champions segment
champion_segment=df[df['RFM Customer Segments']=='Champions']

fig=go.Figure()
fig.add_trace(go.Box(y=champion_segment['RecencyScore'],name='Recency'))
fig.add_trace(go.Box(y=champion_segment['FrequencyScore'],name='Frequency'))
fig.add_trace(go.Box(y=champion_segment['MonetaryScore'],name='Monetary'))
fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)

In [147]:
correlation_matrix=champion_segment[['RecencyScore','FrequencyScore','MonetaryScore']].corr()
heat_map=go.Figure(data=go.Heatmap(
                    z=correlation_matrix.values,
                    x=correlation_matrix.columns,
                    y=correlation_matrix.columns,
                    colorscale='RdBu',
                   colorbar=dict(title='Correlation')))
heat_map.update_layout(title='Correlation Matrix of RFM Values within Champions Segment')

heat_map.show()

In [129]:
import plotly.colors
segment_count=df['RFM Customer Segments'].value_counts().reset_index()
segment_count

fig=go.Figure(data=[go.Bar(x=segment_count['RFM Customer Segments'],y=segment_count['count'],
                           marker=dict(color=pastel_colors))])
fig.update_layout(title='Comparison RFM Segments',
                  xaxis_title='RFM Segment',
                  yaxis_title='Number of Customers' )
fig.show()

In [148]:
df.columns

Index(['CustomerID', 'PurchaseDate', 'TransactionAmount', 'ProductInformation',
       'OrderID', 'Location', 'Recency', 'Frequency', 'MonetaryValue',
       'RecencyScore', 'FrequencyScore', 'MonetaryScore', 'RFM_Score',
       'Value Segment', 'RFM Customer Segments'],
      dtype='object')

In [154]:
segment_score=df.groupby('RFM Customer Segments')[['RecencyScore', 'FrequencyScore', 'MonetaryScore']].mean().reset_index()
fig=go.Figure()
fig.add_trace(go.Bar(
                x=segment_score['RFM Customer Segments'],
                y=segment_score['RecencyScore'],
                name='RecencyScore',
                marker_color='rgb(158,202,225)'
))
fig.add_trace(go.Bar(
                x=segment_score['RFM Customer Segments'],
                y=segment_score['FrequencyScore'],
                name='FrequencyScore',
                marker_color='rgb(94,158,217)'
))
fig.add_trace(go.Bar(
                x=segment_score['RFM Customer Segments'],
                y=segment_score['MonetaryScore'],
                name='MonetaryScore',
                marker_color='rgb(32,102,148)'
))